# g2-forge Quick Start Tutorial

This notebook introduces g2-forge, the universal framework for neural construction of G₂ holonomy metrics.

**What you'll learn:**
1. What are G₂ manifolds?
2. How to configure g2-forge
3. Training a model on custom topology
4. Analyzing training results

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gift-framework/g2-forge/blob/main/notebooks/01_quickstart_tutorial.ipynb)

## 1. Background: G₂ Manifolds

### What is a G₂ manifold?

A **G₂ manifold** is a 7-dimensional Riemannian manifold whose holonomy group is contained in the exceptional Lie group G₂. These manifolds are:

- **Ricci-flat**: Zero Ricci curvature (like Calabi-Yau manifolds)
- **Characterized by a 3-form φ**: The G₂ structure is defined by a special 3-form
- **Important in physics**: Compactifications in M-theory

### The Torsion-Free Conditions

A G₂ structure φ has **torsion-free** (defines a G₂ holonomy metric) if:

$$d\varphi = 0 \quad \text{and} \quad d\star\varphi = 0$$

where ★ is the Hodge star operator.

### Topology: Betti Numbers

G₂ manifolds are characterized by their **Betti numbers**:
- **b₂** = dim H²(M) (harmonic 2-forms)
- **b₃** = dim H³(M) (harmonic 3-forms)

These determine the cohomology and are essential for physics applications.

## 2. Setup

In [ ]:
# Install g2-forge (uncomment for Colab)
# !pip install g2-forge

import torch
import numpy as np
import matplotlib.pyplot as plt

# Check device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
if device == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Import g2-forge
import g2forge as g2
from g2forge.utils.config import (
    G2ForgeConfig,
    TopologyConfig,
    create_k7_config
)
from g2forge.training import Trainer

print(f"g2-forge version: {g2.__version__}")

## 3. Configuration

g2-forge uses a hierarchical configuration system. The key insight is that **everything is parameterized by topology** - the same code works for any (b₂, b₃)!

### Quick Config: GIFT v1.0 Reproduction

In [ ]:
# Load GIFT v1.0 config (b₂=21, b₃=77)
gift_config = G2ForgeConfig.from_gift_v1_0()

print(f"Manifold type: {gift_config.manifold.type}")
print(f"Topology: b₂={gift_config.manifold.topology.b2}, b₃={gift_config.manifold.topology.b3}")
print(f"Euler characteristic: χ = {gift_config.manifold.topology.euler_characteristic}")

### Custom Topology

The power of g2-forge: work with **any topology**!

In [ ]:
# Custom topology: b₂=15, b₃=60
# For TCS: b₂ = b₂(M₁) + b₂(M₂), b₃ = b₃(M₁) + b₃(M₂)
custom_config = create_k7_config(
    b2_m1=8, b3_m1=30,  # Building block M₁
    b2_m2=7, b3_m2=30   # Building block M₂
)

print(f"Custom topology: b₂={custom_config.manifold.topology.b2}, b₃={custom_config.manifold.topology.b3}")
print(f"Networks will auto-size to this topology!")

## 4. Training

Let's train a small model to see the framework in action.

**Note**: Full training takes 15k epochs (~2-6 hours on GPU). Here we'll do a quick demo.

In [ ]:
# For quick demo, use small topology and few epochs
demo_config = create_k7_config(
    b2_m1=3, b3_m1=10,
    b2_m2=2, b3_m2=10
)

# Reduce epochs for demo
demo_config.training.total_epochs = 100
demo_config.training.batch_size = 512

print(f"Demo topology: b₂={demo_config.manifold.topology.b2}, b₃={demo_config.manifold.topology.b3}")

In [ ]:
# Initialize trainer
trainer = Trainer(demo_config, device=device, verbose=True)

# Check network sizes (auto-sized from topology!)
print(f"\nH² network outputs {trainer.h2_network.n_forms} forms (matches b₂={demo_config.manifold.topology.b2})")
print(f"H³ network outputs {trainer.h3_network.n_forms} forms (matches b₃={demo_config.manifold.topology.b3})")

In [ ]:
# Train!
results = trainer.train()

## 5. Analyzing Results

In [ ]:
# Extract training history
history = results['metrics_history']
epochs = [m['epoch'] for m in history]
losses = [m['loss'] for m in history]
torsion_closure = [m['torsion_closure'] for m in history]
torsion_coclosure = [m['torsion_coclosure'] for m in history]

In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Total loss
axes[0].plot(epochs, losses, 'b-', linewidth=1)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Total Loss')
axes[0].set_title('Training Loss')
axes[0].set_yscale('log')
axes[0].grid(True, alpha=0.3)

# Torsion closure (dφ = 0)
axes[1].plot(epochs, torsion_closure, 'r-', linewidth=1)
axes[1].axhline(1e-7, color='g', linestyle='--', label='Target: 10⁻⁷')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Torsion Closure')
axes[1].set_title('dφ = 0 Constraint')
axes[1].set_yscale('log')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Torsion coclosure (d★φ = 0)
axes[2].plot(epochs, torsion_coclosure, 'purple', linewidth=1)
axes[2].axhline(1e-7, color='g', linestyle='--', label='Target: 10⁻⁷')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('Torsion Coclosure')
axes[2].set_title('d★φ = 0 Constraint')
axes[2].set_yscale('log')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_curves.png', dpi=150)
plt.show()

In [ ]:
# Final metrics
final = results['final_metrics']
print("=" * 50)
print("FINAL RESULTS")
print("=" * 50)
print(f"Total Loss:        {final['loss']:.6f}")
print(f"Torsion Closure:   {final['torsion_closure']:.2e}")
print(f"Torsion Coclosure: {final['torsion_coclosure']:.2e}")
print(f"Rank H²:           {final['rank_h2']}/{demo_config.manifold.topology.b2}")
print(f"Rank H³:           {final['rank_h3']}/{demo_config.manifold.topology.b3}")
print(f"Training Time:     {results['training_time_hours']:.2f} hours")

## 6. Spectral Analysis

g2-forge includes tools for analyzing the learned G₂ structure.

In [ ]:
from g2forge.analysis.spectral import (
    extract_harmonic_forms,
    verify_cohomology_ranks
)

# Extract harmonic forms
harmonic_data = extract_harmonic_forms(
    trainer.h2_network,
    trainer.h3_network,
    trainer.manifold,
    n_samples=500,
    device=device
)

print(f"H² orthonormality error: {harmonic_data['orthonormality_error_h2']:.4f}")
print(f"H³ orthonormality error: {harmonic_data['orthonormality_error_h3']:.4f}")
print(f"Computed b₂ rank: {harmonic_data['b2_rank']}")
print(f"Computed b₃ rank: {harmonic_data['b3_rank']}")

In [ ]:
# Verify cohomology ranks
verification = verify_cohomology_ranks(
    harmonic_data['gram_h2'],
    harmonic_data['gram_h3'],
    expected_b2=demo_config.manifold.topology.b2,
    expected_b3=demo_config.manifold.topology.b3
)

print(f"\nCohomology Verification:")
print(f"  b₂ correct: {verification['b2_correct']} ({verification['b2_rank']}/{verification['expected_b2']})")
print(f"  b₃ correct: {verification['b3_correct']} ({verification['b3_rank']}/{verification['expected_b3']})")

## 7. Next Steps

### Full Training

For production results, run 15k epochs:

```python
config = G2ForgeConfig.from_gift_v1_0()  # Or custom topology
trainer = Trainer(config, device='cuda')
results = trainer.train()  # ~2-6 hours on GPU
```

### Expected Results

| Metric | Target | Typical Result |
|--------|--------|----------------|
| Torsion Closure | < 10⁻⁷ | 10⁻⁸ to 10⁻¹¹ |
| Torsion Coclosure | < 10⁻⁷ | 10⁻⁸ to 10⁻¹⁰ |
| Rank H² | b₂ | b₂ (full rank) |
| Rank H³ | b₃ | b₃ (full rank) |

### References

- GIFT Framework: https://github.com/gift-framework/GIFT
- g2-forge Documentation: https://gift-framework.github.io/g2-forge/
- Joyce, D. (2000). *Compact Manifolds with Special Holonomy*

In [ ]:
print("Tutorial complete! Happy G₂ metric construction! 🎉")